## Config

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime, timedelta

In [2]:
from nixtlats import TimeGPT
token = 'REDACTED'

timegpt = TimeGPT(token = token)

timegpt.validate_token()

INFO:nixtlats.timegpt:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

## Read data

In [3]:
df = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3_yearly_processed.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18319 entries, 0 to 18318
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Series    18319 non-null  int64         
 1   N         18319 non-null  int64         
 2   NF        18319 non-null  int64         
 3   Category  18319 non-null  object        
 4   Value     18319 non-null  float64       
 5   Date      18319 non-null  datetime64[us]
dtypes: datetime64[us](1), float64(1), int64(3), object(1)
memory usage: 858.8+ KB


## Function to perform predictions

In [42]:
def simple_forecast_with_timegpt(df):
    forecasts_list = []  # List to hold forecast dataframes for each series
    call_count = 0  # Initialize call count
    start_time = datetime.now()  # Track the start time
    
    for series_id in df['Series'].unique():
        current_time = datetime.now()
        
        # Check if we are approaching the rate limit
        if call_count >= 200:
            # Calculate remaining time to the next minute
            time_to_next_minute = 60 - current_time.second + (1000000 - current_time.microsecond) / 1000000.0
            print(f"Pausing for {time_to_next_minute:.2f} seconds to comply with rate limit.")
            time.sleep(time_to_next_minute)  # Pause execution
            
            # Reset the call count and start time for the new minute
            call_count = 0
            start_time = datetime.now()
        
        series_df = df[df['Series'] == series_id].sort_values('Date')  # Filter and sort by date
        series_df.dropna(subset=['Value'], inplace=True)  # Drop missing values if any
        
        n = int(series_df['N'].iloc[0])
        nf = int(series_df['NF'].iloc[0])
        
        # Ensure the dataframe passed does not use more than (N - NF) examples
        trimmed_df = series_df.head(n - nf)
        
        try:
            forecast_df = timegpt.forecast(trimmed_df, h=nf, id_col='Series', time_col='Date', target_col='Value')
            forecasts_list.append(forecast_df)  # Collect results
        # Call TimeGPT model with nf steps forecast
        except Exception as e:
            print(f"Error encountered for series {series_id}: {e}")
        
        call_count += 1  # Increment call count after each API call
        
        # Check if a minute has passed; if so, reset the counter
        if (datetime.now() - start_time).seconds >= 60:
            call_count = 0
            start_time = datetime.now()


    final_forecasts_df = pd.concat(forecasts_list, ignore_index=True)
    
    return final_forecasts_df

## Run the function

In [43]:
### Took 32 Minutes to run! ###
### Cost 30.00 USD ###
### 645 calls made ###
# all_forecasts = simple_forecast_with_timegpt(df)

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: YS-JAN
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: YS-JAN
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: YS-JAN
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: YS-JAN
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: YS-JAN
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.tim

In [46]:
all_forecasts['Date'] = pd.to_datetime(all_forecasts['Date'])
all_forecasts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3870 entries, 0 to 3869
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Series   3870 non-null   int64         
 1   Date     3870 non-null   datetime64[ns]
 2   TimeGPT  3870 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 90.8 KB


In [47]:
# all_forecasts.to_parquet('M3_yearly_simple_forecasts.parquet', compression='snappy', engine='pyarrow')